<a href="https://colab.research.google.com/github/kilos11/Tensoflow-by-Ricardo-Solinzki/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Formulating the Model**

In [ ]:
import tensorflow as tf


In [ ]:
##Creating variables##
# the following code creates a variable
#named variableA and sets its initial value equal to a tensor named tensorA:
tensorA = tf.constant([1.5, 2.5, 3.5])
variableA = tf.Variable(tensorA)
print(variableA)

#The follow￾ing code creates a variable named variableB and sets its initial value to a tensor
#of normally distributed values:
'''variableB = tf.Variable(tf.random_normal([3]))'''


##Initializing variables##
'''variable_initializer(var_list,
name=‘init’)'''#Returns an operation that initializes the variables in the given list
'''local_variables_initializer()'''# Returns an operation that initializes all local variables
'''global_variables_initializer()'''# Returns an operation that initializes all global variables


##Determining Loss##
"""This method of computing loss is called mean squared error, and it’s one of many
methods available — maximum likelihood estimation and log likelihood estima￾tion are also popular. Chapter 6 discusses statistical regression and the different
ways you can compute loss.
If your model contains neural networks, you can’t compute loss with a simple
equation. Feed-forward networks require a special algorithm like backpropagation,
and recurrent networks rely on backpropagation through time (BPTT). I discuss
neural networks and backpropagation in Chapter 7. I introduce BPTT in Chapter 9.
There’s no right way to compute loss. The only requirement is that every decrease
in loss must imply that the model is closer to the observed data. The process of
improving the model by reducing loss is called optimization."""


##Minimizing Loss with Optimization##
"""After you’ve formed an expression for the loss, the next step is to minimize the
loss by updating the model’s variables. This process is called optimization, and
TensorFlow supports a variety of algorithms for this purpose. Choosing the right
algorithm is critically important when coding machine learning applications.
Each optimization method is represented by a class in the tf.train package. Four
popular optimization classes are the GradientDescentOptimizer, Momentum
Optimizer, AdagradOptimizer, and AdamOptimizer classes. The following
sections look at each of these classes, starting with the Optimizer class, which is
the base class of TensorFlow’s optimization classes.
The Optimizer class
You can’t directly access the Optimizer class in code; applications need to instan￾tiate one of its subclasses instead. But the Optimizer class is crucial because it
defines the all-important minimize method:
minimize(loss, global_step=None, var_list=None, gate_gradients=1, aggregation_
method=None, colocate_gradients_with_ops=False, name=None, grad_loss=None)
The only required argument is the first, which identifies the loss. By default,
minimize can access every trainable variable in the graph. An application can
select specific variables for optimization by setting the var_list argument.
minimize returns an operation that can be executed by a session’s run method.
Each execution performs two steps:
1. Compute values that update the variables of interest.
2. Update the variables of interest with the values computed in Step 1.
Just as you probably won’t win 20 Questions with your first question, you proba￾bly won’t optimize your model with a single call to minimize. Most applications
perform optimization in a loop, and the following code gives an idea what an
optimization loop looks like:

# Create the optimizer and obtain the operation
optimizer = tf.train.GradientDescentOptimizer(learn_rate)
optimizer_op = minimize(loss)
# Execute the minimization operation in a session
with tf.Session() as sess:
 for step in range(num_steps):
 sess.run(optimizer_op)

If the optimizer reaches a suitable minimum, it has converged to the minimum. If
it fails to reach a minimum, the optimizer has diverged.
Each call to the session’s run method minimizes the loss by updating variables.
An application controls how updates are performed by creating a subclass of
Optimizer. This discussion explores four popular Optimizer subclasses: Gradient
DescentOptimizer, MomentumOptimizer, AdagradOptimizer, and AdamOptimizer.
The following discussion gets awfully nerdy, and if you’re just getting started in
machine learning, you don’t really need to know the math. However, selecting the
right optimizer can make a significant impact on the application’s performance.
Also, if you’re interviewing for a lucrative TensorFlow job, you should be able to
justify why you prefer the AdamOptimizer to the GradientDescentOptimizer.

The GradientDescentOptimizer:
The GradientDescentOptimizer is the simplest and most common of the
optimizers used in machine learning. If you look through online example code or
textbooks on machine learning, you’re likely to encounter this optimizer
frequently.
Despite its popularity, few experts recommend the GradientDescentOptimizer
over the alternatives. To see why, you need to understand the algorithm it uses to
perform optimization. In this discussion, I present the theory of gradient descent
and then explain how you can create and use GradientDescentOptimizers in
code.

Creating a GradientDescentOptimizer:
An application can perform optimization with the gradient descent algorithm by
creating a GradientDescentOptimizer. The constructor is given as follows:

tf.train.GradientDescentOptimizer(learning_rate, use_
locking=False, name='GradientDescent')

The learning_rate parameter sets η, the learning rate. The following code cre￾ates an optimizer and sets its learning rate to 0.1:

learn_rate = 0.1
optimizer = tf.train.GradientDescentOptimizer(learn_rate)
optimizer_op = optimizer.minimize(loss)

Many developers set η using trial and error, and initial estimates frequently range
between 0.1 and 0.0001. A common method is to start with a large value of η and
reduce the value until the optimizer converges successfully. Computer scientists
have devised automatic methods for selecting η, but to the best of my knowledge,
no method has gained widespread acceptance.
If you set the use_locking parameter to True, the GradientDescentOptimizer
will acquire a lock that prevents other operations from modifying its variables.
The variables can still be read normally.


The MomentumOptimizer:
The MomentumOptimizer has a lot in common with the GradientDescent
Optimizer, but it usually converges faster with a reduced likelihood of oscillation.
The MomentumOptimizer minimizes loss through the momentum algorithm, which
uses preceding values of the loss gradient to update the current set of variables.
The momentum algorithm introduces a new quantity that TensorFlow calls the
accumulation. This quantity, denoted vt, is determined by the gradient of the cur￾rent loss, the learning rate, and the preceding value of the accumulation:
vt t v J
An application can create a MomentumOptimizer by calling its constructor:

MomentumOptimizer(learning_rate, momentum, use_locking=False, name='Momentum',
use_nesterov=False)

The use_locking parameter has the same purpose as the use_locking parameter
in the GradientDescentOptimizer constructor. That is, the optimizer will lock its
variables’ values if use_locking is set to True.
If use_nesterov is set to True, the optimizer adopts the Nesterov Accelerated
Gradient descent algorithm, which is commonly shortened to NAG. The NAG algo￾rithm modifies the momentum algorithm by updating variables before computing
the loss.


The AdagradOptimizer:
The gradient descent algorithm and the momentum algorithm apply the same
learning rate to each variable being trained. But different variables may converge
to their minima at different rates. The adaptive gradient (Adagrad) algorithm
takes this into account.
The Adagrad algorithm has two characteristics that have made it popular among
academics and experts:
» The learning rate changes from variable to variable and from step to step. The
learning rate at the tth step for the ith variable is denoted t i, .
» Adagrad methods compute subgradients instead of gradients. A subgradient is
a generalization of a gradient that applies to nondifferentiable functions. This
means AdaGrad methods can optimize both differentiable and nondifferen￾tiable functions.
Thankfully, TensorFlow developers don’t have to worry about subgradients or
outer products. This is because the TensorFlow API provides the Adagrad
Optimizer class, whose constructor is given as follows.

AdagradOptimizer(learning_rate, initial_accumulator_value=0.1, use_locking=False,
name='Adagrad')

One shortcoming of the Adagrad algorithm is that the learning rates always
decrease in magnitude. As training continues, their values will eventually reach
zero, bringing training to a halt.


The AdamOptimizer:
The Adam (Adaptive Moment Estimation) algorithm closely resembles the Adag￾rad algorithm in many respects.
It also resembles the Momentum algorithm
because it takes two factors into account:
» The first moment vector: Scales the gradient by 1 -1
» The second moment vector: Scales the square of the gradient by 1 -2
To employ the Adam algorithm, you need to create an instance of AdamOptimizer.
The constructor is given as follows:

AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08,
use_locking=False, name='Adam')

When deciding on an optimizer, I always start with the AdamOptimizer, especially
when working with images. """






<tf.Variable 'Variable:0' shape=(3,) dtype=float32>


## **Feeding Data into a Session**

In [ ]:
'''Instead of processing all the test data with one call to a session’s run method,
applications frequently split the data into portions and call run once for each por￾tion. There are at least three reasons to do so:
» If the data is stored in a file or on a remote server, it may be more efficient to
process one portion of data while another is loaded from the source.
» Shuffling the portions of data increases the data’s stochasticity. This process
can improve convergence to a global minimum instead of a local minimum.
I explain the rationale for data shuffling in the upcoming “Stochasticity” section.
» Time constraints make it impractical to process all the data at once.
A portion of data processed in one session execution is called a batch. The process
of transferring batches to a session is called feeding data to the session. To configure
this in code, an application needs to perform three steps:
1. Define placeholders to contain the data to be fed into the session.
2. Use the placeholders in the expressions for model and loss.
3. Set the second parameter of the session’s run method to a dictionary
that associates each placeholder with a source of data.
Step 2 is trivial because you can process placeholders in the same way that you can
process tensors. This discussion focuses on Steps 1 and 3. Later chapters present
code that demonstrates how data can be fed into a session.

Creating placeholders:
A placeholder is a constant Tensor that holds a batch of data to be fed into a
session. You can create placeholders by calling the tf.placeholder function:

tf.placeholder(dtype, shape=None, name=None)

The first two arguments specify the type of the placeholder’s elements and its
size. The actual content of a placeholder is set by the running session, so there’s
no way to initialize a placeholder.
For example, the following code creates a placeholder that contains 32-bit
floating-point values:

ph = tf.placeholder(tf.float32)

If a placeholder’s shape isn’t given, it can be set to a tensor of any shape. If the
shape is given, assigning a tensor of a different shape will cause an error.
When associating data with a placeholder, there’s a catch: The data source can be
a list of constants or a NumPy ndarray, but it can’t be a tensor. The following code
associates a placeholder with an ndarray:'''

ph = tf.placeholder(tf.float32)
vals = np.array([9., 8., 7.])
incr = tf.add(ph, 1.)
with tf.Session() as sess:
 res = sess.run(incr, feed_dict={ph: vals})
 print(res)

 '''Find name of all students in the table,
 Fetch current date-time from the system, create a table called heroes'''


 '''Stochasticity:
 To keep optimizers from converging to a local minimum instead of a global mini￾mum, many applications split their training data into small batches and feed them
randomly to the session. This randomness, also called stochasticity, forces the opti￾mizer to take larger jumps at first and smaller jumps as training progresses. This
jumping increases the likelihood that the optimizer will find a global minimum.
If the gradient descent algorithm is employed to process stochastic data, it’s
referred to as the stochastic gradient descent algorithm. If you encounter the term
SGD in machine learning literature, this algorithm is what it’s referring to.'''













## Monitoring Steps, Global Steps,
and Epochs:


In [ ]:
'''Monitoring Steps, Global Steps,
and Epochs:
In TensorFlow, each session execution that processes a single batch of data is
called a step. Many TensorFlow functions and methods accept a parameter called
global_step, which can be used to monitor the total number of steps executed by
a session. In practice, global_step serves as the index of the batch being pro￾cessed. You can access this index in code by calling tf.train.global_step.
You can also store the global step in a regular variable. This storage requires two
operations:
1. Define a variable with an initial value of 0 and its trainable argument
set to False.
2. Set the variable equal to the global_step parameter of the optimizer’s
minimize method.
If its global_step parameter is set to a variable, minimize will increment the
variable each time a session processes a batch of data. The following code creates
a variable named gstep and configures it to store the application’s global step:'''

# Define the variable to hold the global step
gstep = tf.Variable(0, trainable=False)
# Configure the optimizer
learn_rate = 0.1
batch_size = 40
optimizer = tf.train.GradientDescentOptimizer(learn_rate).
 minimize(loss, global_step=gstep)
# Initialize variables
init = tf.global_variables_initializer()
# Launch session
with tf.Session() as sess:
 sess.run(init)
 for batch in range(batch_size):
 _, step, result = sess.run([optimizer, gstep, x_min])
 print("Step %d: Computed result = %f" % (step, result))

 '''As you look at this code, a question may occur to you: Why keep track of the global
step when you can access the loop index? To answer this question, suppose that
you execute ten training batches and then restart your application. The loop vari￾able will revert back to 0, but if you’d saved the global step to a file, you can
restore it and use it as the current global step. I explain how to save variables to a
file in the section “Saving variables,” later in this chapter.
In the preceding example, the test executes each batch only once. In a real-world
application, all the batches will be processed multiple times. A pass through every
batch of a dataset is referred to as an epoch. For example, if a dataset is split into
50 batches, an epoch consists of 50 steps.
Many applications execute sessions in two loops: The outer loop iterates once for
each epoch, and the inner loop executes once for each batch. The following code
creates the two loops and calls sess.run with each iteration:'''

for epoch in range(epochs):
 for batch in range(batch_size):
 _, step, result = sess.run([optimizer, gstep, x_min])
 print("Step %d: Computed result = %f" % (step, result))

## **Saving and Restoring Variables**

In [ ]:
'''The Saver class makes it straightforward to load and store variables. By default, a
Saver can access every variable in the session. But the first argument of the con￾structor can identify specific variables to be accessed. For example, the following
code creates a Saver that can save/restore only two variables: firstVar and
secondVar:'''

saver = tf.train.Saver([firstVar, secondVar])

'''After you create a Saver, you can store variables to a file by calling its save method.
Then you can restore variables from a file by calling restore.'''

'''Saving variables:
The save method stores variables and data related to the variables. By default, the
method creates at least three binary files, each with the same name but a different
suffix:
» filename.data-X-of-Y: Stores variable values
» filename.index: Holds the offset of each variable in the data file(s)
» filename.meta: MetaGraphDef containing the structure of the graph that
contains the variables The data files contain variable values, and if the application has many variables,
save will create multiple data files. If there’s only one file, its name will be
filename.data-00000-of-00001.
The index file contains a table that matches variable names to offsets in the index
file. You can retrieve variables using the restore method, which I explain in the
next section.
You can create these files by creating a Saver and calling its save method:'''

save(sess, save_path, global_step=None, latest_filename=None,
meta_graph_suffix='meta', write_meta_graph=True, write_
state=True)

'''These parameters are straightforward to understand. sess is the session contain￾ing the variables of interest and save_path identifies the path of the file to contain
the saved data. The last element of save_path specifies the name of the files to be
generated.
If latest_filename is set, save will create a text file that lists the paths of files
involved in the save operation. If global_step is set, the value will be appended
to each of the generated files.
For example, the following code creates a Saver and calls save to create the
generated files (output.*) in the current directory:'''

saver = tf.train.Saver()
saver.save(sess, os.getcwd() + "/output")

'''If there aren’t many variables to store, save will generate only three files:
output.data-00000-of-00001, output.index, and output.meta.'''

'''Restoring variables:
The restore method loads variables that have been stored previously. The process
of restoring variables consists of two steps:
1. Call import_meta_graph to add the variables’ nodes to the current graph.
2. Call restore to access the variable data.
The first step is simple. tf.train.import_meta_graph accepts the path of a
*.meta file, reads graph data from the file, and adds the graph’s nodes to the cur￾rent graph. The function returns a Saver that lets you restore variables from the
loaded graph.
For example, the following code imports graph data from output.meta and
obtains a Saver that can be used to load variables:'''

saver = tf.train.import_meta_graph("output.meta")

'''After obtaining the Saver, an application can load variables by calling its restore
method, whose signature is given as follows:'''

restore(sess, save_path)

'''As in the save method, sess identifies the session containing the variables, and
save_path is the path to the file containing the variable data. This path must
include the name of the three files without the suffix. As an example, the follow￾ing
code uses saver to load variables from output into the current graph:'''

saver.restore(sess, os.getcwd() + "/output")



## **Working with SavedModels**

In [ ]:
'''In addition to storing variables with a Saver, you can store your application’s entire
model by creating a SavedModel. As stated in the documentation, SavedModels are
“the universal serialization format for TensorFlow models” and serve as “the
canonical way to export TensorFlow graphs.”
To be precise, a SavedModel is a directory that contains a *.pb or *.pbtxt file. This
file contains the application’s model and stores graph definitions in MetaGraphDef
protocol buffers. In addition to this file, a SavedModel may contain one or more of
the following subdirectories:
» variables: A directory containing the application’s variables (files are similar to
those produced by the Saver’s save method, excluding the *.meta file)
» assets: Auxiliary files that need to be loaded into the graph
» assets.extra: User-provided files that don’t need to be loaded into the graph
Saving and restoring a SavedModel isn’t conceptually difficult, but the code gets a
little complicated.'''

'''Saving a SavedModel:
The process of saving an application’s model to a SavedModel is similar to the
process of storing variables. But instead of creating a Saver, you need to create a
tf.saved_model.builder.SavedModelBuilder. The constructor accepts a single
argument that identifies the top-level directory:'''

builder = tf.saved_model.builder.SavedModelBuilder("out")

'''After creating a SavedModelBuilder, you can add data to the model and save the
model to the given directory. To add data to the model, you need to call one of two
functions: add_meta_graph or add_meta_graph_and_variables. The signature of
add_meta_graph is given as'''

add_meta_graph(tags, signature_def_map=None, assets_
collection=None, legacy_init_op=None, clear_devices=False,
main_op=None)

'''Metagraphs identify their capabilities and purposes with strings called tags. You
can assign a metagraph’s tags by setting the tags parameter. The tf.saved_
model.tag_constants provides three common tags: GPU, SERVING, and TRAINING.
A graph’s inputs and outputs form its signature. In code, a graph’s signature is
represented by a SignatureDef, and you can create this by calling the build_
signature_def function of the tf.saved_model.signature_def_utils package:'''

build_signature_def(inputs=None, outputs=None, method_name=None)

'''To create the signature, you need to set inputs and outputs to dictionaries that
associates names with TensorInfo protocol buffers. For the names, many appli￾cations use constants from tf.saved_model.signature_constants, which
include CLASSIFY_INPUTS, CLASSIFY_OUTPUT_CLASSES, PREDICT_INPUTS, and
PREDICT_OUTPUTS.
You can obtain a TensorInfo for a tensor by calling tf.saved_model.utils.
build_tensor_info with the tensor. The following code returns a TensorInfo for
a tensor named input_vec:'''

info = tf.saved_model.utils.build_tensor_info(input_vec)

'''The method_name parameter of build_signature_def is a string that serves as
the signature’s method name. You can set this to one of the strings in the
tf.saved_model.signature_constants module, such as CLASSIFY_METHOD_
NAME, PREDICT_METHOD_NAME, or REGRESS_METHOD_NAME.
The add_meta_graph_and_variables method is similar to add_meta_graph, but it
has an extra parameter. The first parameter of add_meta_graph_and_variables is
sess, which identifies the session that should provide the metagraph’s variables.
After you’ve added metagraphs to a SavedModel, you can store the SavedModel by
calling the save method. This accepts an as_text parameter that identifies
whether the protocol buffer should be saved as a text file (*.pbtxt) or a binary file
(*.pb). By default, save stores metagraph data in a binary file.'''

'''Loading a SavedModel:
While it’s complex to save metagraphs to a SavedModel, it’s easy to load them.
You need to know only one function:'''

tf.saved_model.loader.load(sess, tags, export_dir, **saver_kwargs)

'''This loads the MetaGraphDef protocol buffer from the directory given by export_
dir with the tags given by tags. The sess parameter identifies the session that
should contain the metagraph’s variables, assets, and signatures.'''


## **Putting Theory into Practice**

In [ ]:
#Training in TensorFlow
# Define a trainable variable
x_var = tf.Variable(0., name='x_result')

# Define an untrainable variable to hold the global step
step_var = tf.Variable(0, trainable=False)

# Express loss in terms of the variable
loss = x_var * x_var - 4.0 * x_var + 5.0

# Find variable value that minimizes loss
learn_rate = 0.1
num_epochs = 40
optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss, global_
step=step_var)

# Initialize variables
init = tf.global_variables_initializer()

# Create the saver
saver = tf.train.Saver()

# Create summary data and FileWriter
summary_op = tf.summary.scalar('x', x_var)
file_writer = tf.summary.FileWriter('log', graph=tf.get_default_graph())

# Launch session
with tf.Session() as sess:
 sess.run(init)
 for epoch in range(num_epochs):
    _, step, result, summary = sess.run([optimizer, step_var, x_var,
summary_op])
    print('Step %d: Computed result = %f' % (step, result))

    # Print summary data
    file_writer.add_summary(summary, global_step=step)
    file_writer.flush()
# Store variable data
 saver.save(sess, os.getcwd() + '/output')
 print('Final x_var: %f' % sess.run(x_var))

 '''This code creates two variables: a trainable variable named x_var and an untrain￾able variable named step_var. loss is set to a quadratic equation whose indepen￾dent variable is x_var.
The application calls tf.global_variables_initializer to obtain an operation
for initializing its variables. The session must execute this operation before it can
train the variables in the optimization process.
After creating the variables, the application creates a GradientDescentOptimizer
and calls its minimize method to reduce the loss to a minimum. Then it assigns
the global_step parameter of minimize to step_var. This assignment tells the
session to increment step_var each time it performs a training operation.
After each training operation, print displays the global step and the current value
of x_var. As training continues, x_var approaches 2, the point of minimum
loss.  Similarly, step_var approaches 40 because the training loop performs
40 iterations.
When optimization is complete, the application stores its variables to a file. The
save method of the Saver instance stores variable data to three files in the current
directory: output.data-00000-of-00001, output.index, and output.meta.
The code in ch5/restore_vars.py loads the value of x_var from the new data
files. Listing 5-2 presents the code.'''

#Loading Variables from a File
# Create session
with tf.Session() as sess:
 # Load stored graph into current graph
 saver = tf.train.import_meta_graph('output.meta')
 # Restore variables into graph
 saver.restore(sess, os.getcwd() + '/output')
 # Display value of variable
 print('Variable value: ', sess.run('x_result:0'))

 '''It’s important to see that this code doesn’t create a Saver by calling the class’s
constructor. Instead, it obtains a Saver by calling import_meta_graph with the
name of the file containing graph data.
After obtaining the Saver, the application obtains the variable’s value by calling
the Saver’s restore method and the session’s run method. Even though the vari￾able’s name was x_var, the application calls run with x_output:0 because the
variable’s name parameter was set to x_output.'''


## **Visualizing the Training Process**

In [ ]:
#The ch5/simple_train.py application prints the loss at each step using the fol￾lowing code:
print("Step %d: Computed result = %f" % (step, result))

'''TensorFlow provides a better way to monitor training. Chapter  4 covers the
TensorBoard utility, which reads summary data generated by an application. The
code in ch5/simple_train.py generates summary data for training by perform￾ing four steps:
1. Call tf.summary.scalar to create an operation that writes x_var to
summary data.
2. Call tf.summary.FileWriter to create a FileWriter.
3. Execute the session with the operation from Step 1.
4. With each session execution, print the summary data by calling the
FileWriter’s add_summary method'''
#For the last step, the following code prints the summary data:
file_writer.add_summary(summary, global_step=step)
